## Linear Programming in Python - Land Use Example

Let's solve the land use transformation problem using Python.

The method is the same as for the CASA merchandising problem, so you should look at that example first.

In [1]:
import scipy.optimize as spo
# The documentation for scipy.optimize.linprog is here:
# https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linprog.html

#### This is the problem we are trying to solve...

![farm_land](https://github.com/huanfachen/Quantitative_Methods_CASA0007/blob/main/images/geograph-2132256-by-Richard-Croft.jpg?raw=true)

An area of land is available for reclassification. 8650 ha of the land is forest (F), 3875 is rainfed cropland (R), and 1375 ha is irrigated cropland (I). Land may be reclassified as one of these types or as urban land (U).

A budget of 25000 pesetas is available for transforming the land. The costs in pesetas to transform an area of 10 ha are as follows:


| From / To |    F |    R |     I |      U |
| --------: | ---: | ---: | ----: | -----: |
|         F |   NA | 7.20 |  51.2 | 750.00 |
|         R | 1.80 |   NA | 16.20 | 625.00 |
|         I |   NA |   NA |    NA | 625.00 |

Transformations marked with 'NA' are not possible.

Not all the existing land is suitable for transformation to all other types.

- 21 ha of forest land and 219 ha of rainfed cropland are suitable for tranformation to irrigated cropland.
- 369 ha of forest, 882 ha of rainfed cropland and 205 ha of irrigated cropland are suitable for transformation to urban land.

Additionally, for environmental reasons, at least 75% of existing forest land must be preserved.

The principal objective of the reclassification programme is to create jobs:

- 10 ha of forest supports 0.017 jobs;
- 10 ha of rainfed cropland supports 0.5 jobs;
- 10 ha of irrigated cropland supports 2 jobs;
- 10 ha of urban land supports 18 jobs.

How should the land be reallocated to maximise employment?

(Adapted from Chuvieco, E. (1993), 'Integration of linear programming and GIS for land-use modelling')

#### And here is the problem written as a linear program:

Maximise:  
E = 0.017 FF + 0.5 FR + 2 FI + 18 FU + 0.017 RF + 0.5 RR + 2 RI + 18 RU + 2 II + 18 IU

Subject to:  
**A)** FF + FR + FI + FU = 865.00  
**B)** RF + RR + RI + RU = 387.50  
**C)** II + IU = 137.50  
**D)** FU ≤ 36.90  
**E)** RU ≤ 88.20  
**F)** IU ≤ 20.50   
**G)** FI ≤ 02.10  
**H)** RI ≤ 21.90  
**I)** FF ≥ 648.75  
**J)** 7.2 FR +51.2 FI + 750 FU + 1.8 RF + 16.2 RI + 625 RU + 625 IU ≤ 25000  
**K)** FF, FR, FI, FU, RF, RR, RI, RU, II, IU ≥ 0

Where:  
- XY = Number of 10 ha units of terrain type X converted to terrain type Y
- XX = Number of 10 ha units of terrain type X kept as terrain type X
- E = Employment (total jobs supported by all land after transformation)

Some notes on the objective function - after re-classification, the amount of each terrain type is:
- Forest: (FF + RF)
- Irrigated cropland: (II + FI + RI)
- Rainfed cropland: (RR + FR)
- Urban: (FU + RU + IU)

In [5]:
# First the coefficients of the objective function.
# Note the order of the variables in the objective function and coefficients are:
# FF, FR, FI, FU, RF, RR, RI, RU, IF, IR, II, IU
# This order is not explicitly stated, but is used for objective function and constraints.
# Since this is a MAXIMISATION problem, while Python expects a MINIMISATION, the objective coefficients change sign:

# E = 0.017 FF + 0.5 FR + 2 FI + 18 FU + 0.017 RF + 0.5 RR + 2 RI + 18 RU + 2 II + 18 IU
# Note the order of variables: FF, FR, FI, FU, RF, RR, RI, RU, II, IU
objective_coeffs = [-0.017, -0.5, -2, -18, -0.017, -0.5, -2, -18, -2, -18]

# Next the constraints.
# We note that many of the constraints (including the non-negativity constraints)...
# ... can be covered by setting upper and lower bounds on each variable as follows:

FF_bounds = (648.75,None)
FR_bounds = (0,None)
FI_bounds = (0,2.10)
FU_bounds = (0,36.90)
RF_bounds = (0,None)
RR_bounds = (0,None)
RI_bounds = (0,21.90)
RU_bounds = (0,88.20)
II_bounds = (0,None)
IU_bounds = (0,20.50)

# It will be handy to collect all these bounds together in a tuple (like a list, but defined with round brackets):

all_bounds = (FF_bounds,
              FR_bounds,
              FI_bounds,
              FU_bounds,
              RF_bounds,
              RR_bounds,
              RI_bounds,
              RU_bounds,
              II_bounds,
              IU_bounds)

# The only constraints that cannot be dealt with in this way...
# ... (since they feature more than one decision variable) are A, B, C and J.

# Note also that A, B and C are EQUALITY constraints, rather than INEQUALITY constraints.
# These must be dealt with separately.

# Here are the coefficients for the equality constraints (A, B, C) and the corresponding constants:
# FF + FR + FI + FU = 865.00  
# RF + RR + RI + RU = 387.50  
# II + IU = 137.50 
# Note the order of variables: FF, FR, FI, FU, RF, RR, RI, RU, II, IU
# Zeros are used when a variable is not involved in a constraint

eq_constraint_coeffs = [[1,1,1,1,0,0,0,0,0,0],
                        [0,0,0,0,1,1,1,1,0,0],
                        [0,0,0,0,0,0,0,0,1,1]]
eq_constraint_consts = [865,387.5,137.5]

# And here are the coefficients for the inequality constraint (J) and the corresponding constant:

ineq_constraint_coeffs = [[0,7.2,51.2,750,1.8,0,16.2,625,0,625]]
ineq_constraint_consts = [25000]

# Note that even though there is only one inequality constraint, these values are still presented in lists.

# Make sure everything stays in the right order!

In [4]:
# Now we perform the optimisation:

# shortcut of opening documentation in Jupyter lab: SHIFT + TAB
results = spo.linprog(objective_coeffs, A_eq=eq_constraint_coeffs, b_eq=eq_constraint_consts, A_ub=ineq_constraint_coeffs, b_ub=ineq_constraint_consts, bounds=all_bounds,options={"disp": True})

# And report the optimal value of each variable:
FF,FR,FI,FU,RF,RR,RI,RU,II,IU = results['x']
print("FF =", FF)
print("FR =", FR)
print("FI =", FI)
print("FU =", FU)
print("RF =", RF)
print("RR =", RR)
print("RI =", RI)
print("RU =", RU)
print("II =", II)
print("IU =", IU)
print()

# And the optimal value of the objective function:
E = -results['fun']
print("E =", E)


Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -72.06275           
0.1671779194894     0.1671779194894     0.1671779194895     0.8377344156294  0.1671779194894     -117.9687531267     
0.02157489805608    0.02157489805608    0.02157489805609    0.9183422109627  0.02157489805608    -514.7074151919     
0.003732612219206   0.003732612219206   0.003732612219207   0.8347019077551  0.003732612219206   -1007.039937308     
0.0003817839899822  0.000381783989984   0.0003817839899841  0.9103294229511  0.0003817839899872  -1236.061857436     
3.208955794562e-05  3.208955794584e-05  3.208955794583e-05  0.917769461251   3.20895579463e-05   -1264.906433482     
5.170714592002e-07  5.170714592869e-07  5.170714594853e-07  1.0              5.170714593264e-07  -1267.747431067     
2.68387328342e-11   2.683876353034e-11  2.683868829392e-1

In [4]:
# Let's report on our results:
print("The optimal reclassification plan is to transform:")
print("*", int(round(FR*10,0)), "ha of forest to rainfed cropland,", int(round(FI*10,0)), "ha to irrigated cropland, and", int(round(FU*10,0)), "ha to urban land.")
print("*", int(round(RF*10,0)), "ha of rainfed cropland to forest,", int(round(RI*10,0)), "ha to irrigated cropland, and", int(round(RU*10,0)), "ha to urban land.")
print("*", int(round(IU*10,0)), "ha of irrigated cropland to urban land.")
print("The land will then support", int(round(E,0)), "jobs.")

The optimal reclassification plan is to transform:
* 2141 ha of forest to rainfed cropland, 21 ha to irrigated cropland, and 0 ha to urban land.
* 0 ha of rainfed cropland to forest, 219 ha to irrigated cropland, and 368 ha to urban land.
* 0 ha of irrigated cropland to urban land.
The land will then support 1268 jobs.


In [5]:
# We can compare this to the current number of jobs supported by the land to see how many jobs could be created:
current_jobs = -objective_coeffs[0]*eq_constraint_consts[0] - objective_coeffs[1]*eq_constraint_consts[1] - objective_coeffs[2]*eq_constraint_consts[2]
print("The land currently supports", int(round(current_jobs,0)), "jobs.")
print("This means that", int(round(E,0)-round(current_jobs,0)), "jobs could be created with this reclassification scheme.")

The land currently supports 483 jobs.
This means that 785 jobs could be created with this reclassification scheme.
